In [1]:
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


# функция, создающая объект класса cls и возвращающая свертку от input_matrix
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
# Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()).reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) and (custom_conv2d_out.shape == conv2d_out.shape)

print(test_conv2d_layer(Conv2d))

True


In [ ]:
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
## Абстрактный слой в данном случае нужен для того, чтобы не перезадавать переменные
## Если мы делаем 2 разных класса, которые делают что-то похожее,
##  то общую часть можно записать здесь и при создании нового класса наследовать этот
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel
## ниже указан декоратор @abstractmethod. Это значит, что при создании 
## нового класса на основании этого функцию нужно будет переопределить
## иначе будет ошибка
    @abstractmethod
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
## Это класс который выполняет свёртку двумерного слоя
## согласно документации https://pytorch.org/docs/stable/nn.html#conv2d
## выполняется это следующим образом. Сам conv2d это класс. 
## Для начала нужно передать ему параметры. После этого вызвать как функцию 
## с матрицей, которую нужно свернуть. 
## Не смог найти в документации это - self.conv2d.weight.data = kernel
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)
     


    # функция, создающая объект класса cls и возвращающая свертку от input_matrix
## Эта функия подготоавливает данные и вызывает нужный класс для свертки
## Сделано это для упрощения. 
## Обратите внимание самым первым параметром указывается
## класс, которым нужно свернуть матрицу   conv2d_layer_class

def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
## Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
## Функция тестирует conv2d_layer_class. 

def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    print('Вызвана функция test_conv2d_layer ей передан на тестирование класс ',conv2d_layer_class)
    
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])
    print('Kernel с помощю которого будет выполняться сворачивание \n', kernel)
    in_channels = kernel.shape[1]

## создается тензор размерности 6 таблиц 4*4 
## !Comment: "только в комменте ошибка: "тензор размерности 6 таблиц 4*4" - размерность input_tensor будет 2*3*4*4."
## (то есть высота- 4, ширина -4) количество каналов - 3 (берется из размера kernel)
## и 2 батча 2*3=6
    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()).reshape(batch_size, in_channels, input_height, input_width)
    print('Матрица, которую надо будет свернуть \n', input_tensor)
 ## Здесь вызываются 2 фукнции, которые выполняют сворачивание
 ## Одной передается класс conv2d_layer_class другой Conv2d
 ## Обратите внимание что conv2d_layer_class это аргумент текущей функции 
 ## Эту функцию вызвали через print(test_conv2d_layer(Conv2d))
 ## то есть по факту сравнивается Conv2d и Conv2d
 
    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    print('Класс ',conv2d_layer_class,' выдает свернутую матрицу \n',custom_conv2d_out)

    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    print('Класс ',Conv2d,' выдает свернутую матрицу \n',conv2d_out)

    return torch.allclose(custom_conv2d_out, conv2d_out) and (custom_conv2d_out.shape == conv2d_out.shape)

print(test_conv2d_layer(Conv2d))

Вызвана функция test_conv2d_layer ей передан на тестирование класс  <class '__main__.Conv2d'>
Kernel с помощю которого будет выполняться сворачивание 
 tensor([[[[ 0.,  1.,  0.],
          [ 1.,  2.,  1.],
          [ 0.,  1.,  0.]],

         [[ 1.,  2.,  1.],
          [ 0.,  3.,  3.],
          [ 0.,  1., 10.]],

         [[10., 11., 12.],
          [13., 14., 15.],
          [16., 17., 18.]]]])
Матрица, которую надо будет свернуть 
 tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]],


        [[[48., 49., 50., 51.],
          [52., 53., 54., 55.],
          [56., 57., 58., 59.],
          [60., 61., 62., 63.]],

         [[64., 65., 66., 67.],
     